<a href="https://colab.research.google.com/github/jonathas-1993/Topicos_UFAM/blob/main/3_Preparing_IMDB_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Preparando a base IMDM para processamento por Bag of Words, sequences and transformers.

Vamos começar baixando a base de dados da página de Andrew Maas de Standofrd e descomprimindo ela.

In [ ]:
!curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz # opção -O salva arquivo com o mesmo nome no
!tar -xf aclImdb_v1.tar.gz                                               # diretório atual. opção - o permite especificar
                                                                         # outro nome

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 80.2M  100 80.2M    0     0  22.0M      0  0:00:03  0:00:03 --:--:-- 22.0M


For instance, the train/pos/ directory contains a set of 12,500 text files, each of which contains the text body of a positive-sentiment movie review to be used as training data.The negative-sentiment reviews live in the “neg” directories. In total, there are 25,000 text files for training and another 25,000 for testing.

Take a look at the content of a few of these text files. Whether you’re working withtext data or image data, remember to alwaysinspect what your data looks like beforeyou dive into modeling it. It will ground your intuition about what your model is actu-
ally doing:

In [ ]:
!cat aclImdb/train/pos/4077_10.txt      # o comando  cat mostra o conteúdo de um arquivo

I first saw this back in the early 90s on UK TV, i did like it then but i missed the chance to tape it, many years passed but the film always stuck with me and i lost hope of seeing it TV again, the main thing that stuck with me was the end, the hole castle part really touched me, its easy to watch, has a great story, great music, the list goes on and on, its OK me saying how good it is but everyone will take there own best bits away with them once they have seen it, yes the animation is top notch and beautiful to watch, it does show its age in a very few parts but that has now become part of it beauty, i am so glad it has came out on DVD as it is one of my top 10 films of all time. Buy it or rent it just see it, best viewing is at night alone with drink and food in reach so you don't have to stop the film.<br /><br />Enjoy

There’s also a train/unsup subdirectoryin there, which we don’t need. Let’s delete it: (não pode ter outras pastas além
daquelas contendo os exemplos das classes)

In [ ]:
!rm -r aclImdb/train/unsup

Next, let’s prepare a validation set by setting apart 20% of the training text

1.   Item de lista
2.   Item de lista

files in a
new directory, aclImdb/val:

In [ ]:
import os, pathlib, shutil, random

In [ ]:
base_dir = pathlib.Path("aclImdb")
val_dir = base_dir /"val"
train_dir = base_dir /"train"
for category in("neg","pos"):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)           #Shuffle the list of training files using a seed, to ensure we get
                                                 # the same validation set every time we run the code.
    num_val_samples = int(0.2* len(files))       #Take 20% of the training files to use for validation
    val_files=files[-num_val_samples:]
    for fname in val_files:
      shutil.move(train_dir/category/fname,      #The shutil module offers a number of high-level operations on files
                  val_dir/category/fname)        #Move the files to aclImdb/val/neg and aclImdb/val/pos.

Let’s create threeDatasetobjects for training, validation, and testing:

*   Item de lista
*   Item de lista





In [ ]:
from tensorflow import keras
batch_size=32

In [ ]:
train_ds = keras.utils.text_dataset_from_directory(    #main_directory/
"aclImdb/train", batch_size=batch_size                 #...class_a/
)                                                      #......a_text_1.txt
val_ds = keras.utils.text_dataset_from_directory(      #......a_text_2.txt
"aclImdb/val", batch_size=batch_size                   #...class_b/
)                                                      #......b_text_1.txt
test_ds = keras.utils.text_dataset_from_directory(     #......b_text_2.txt
"aclImdb/test", batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


Running this line should output “Found 20000 files belonging to 2 classes”; if you see “Found 70000 files belonging to 3 classes,” it means you forgot to delete the aclImdb/train/unsup directory.

These datasets yield inputs that are TensorFlow tf.string tensors and targets that are
int32 tensors encoding the value “0” or “1.”

Displaying the shapes and dtypes of the first batch

In [ ]:
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break


inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b"Okay, that was a pretty damn good episode. Much better than the credit it receives.<br /><br />The camera work is splendid. Best yet. I love that final shot. The atmosphere is fantastic, the costumes are great and the guest cast (minus the helpless victims) is strong. <br /><br />What I don't like about this episode is that many things that are left unexplained. why does it change sex? what's the purpose? and they're aliens? what kind? why were they never shown again in the later mythology?<br /><br />I'm giving this episode a high THREE stars. One of my favorites yet, but the plot holes bother me. Still... not gonna let it ruin my entertainment.", shape=(), dtype=string)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


"********************************************************"-
***O código a Seguir refere-se a "*TextVectorization_bag_unigram
"********************************************************"

Limit the vocabulary to the 20,000 most frequent words.Otherwise we’d be indexing every word in the training data—potentially tens of thousands of terms that only occur once or twice and thus aren’t informative. In general, 20,000 is the right vocabulary size for text classification.

In [ ]:
from tensorflow.keras.layers import TextVectorization

In [ ]:
text_vectorization = TextVectorization(
    max_tokens=20000,
    output_mode="multi_hot",                #Encode the output tokens as multi-hot binary vectors.
)


:map(
    map_func, num_parallel_calls=None, deterministic=None, name=None
)
Maps map_func across the elements of this dataset.

In [ ]:
text_only_train_ds = train_ds.map(lambda x, y: x)  #Prepare a dataset that only yields raw text inputs (no labels).
text_vectorization.adapt(text_only_train_ds) #Use that dataset to index the dataset vocabulary via the adapt() method

binary_1gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),num_parallel_calls=4) #Prepare processed versions of our training,
binary_1gram_val_ds = val_ds.map(                                 #validation, and test dataset.
    lambda x, y: (text_vectorization(x), y),num_parallel_calls=4) #Make sure to specify num_parallel_calls
binary_1gram_test_ds = test_ds.map(                               #to leverage multiple CPU cores.
    lambda x, y: (text_vectorization(x), y),num_parallel_calls=4) # o resultado de lambda mapeia x,y em vetor multi-hot,y

Inspecting the output of our binary unigram dataset

In [ ]:
for inputs, targets in binary_1gram_train_ds:
  print("inputs.shape:", inputs.shape)   #Inputs are batches of 20,000-dimensional vectors.
  print("inputs.dtype:", inputs.dtype)   #These vectors consistentirely of ones and zeros.
  print("targets.shape:", targets.shape)
  print("targets.dtype:", targets.dtype)
  print("inputs[0]:", inputs[0])
  print("targets[0]:", targets[0])
  break

inputs.shape: (32, 20000)
inputs.dtype: <dtype: 'int64'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor([1 1 1 ... 0 0 0], shape=(20000,), dtype=int64)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


Criando um modelo simples de classificação com uma camada escondida e uma camada de saída. Funções ReLU e sigmmoid. Usa Dropout.

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
def get_model(max_tokens=20000, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
    loss="binary_crossentropy",
    metrics=["accuracy"])
    return model

Treinando o modelo (We call cache() on the datasets to cache them in memory: this way, we will only do the preprocessing once, during the first epoch, and we’ll reuse the preprocessed texts for the following epochs. This can only be done if the data is small enough to fit in memory.)

In [ ]:
model = get_model()
model.summary()
checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/tmp/checkpoint.weights.h5',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
model.fit(binary_1gram_train_ds,
validation_data=binary_1gram_val_ds.cache(),epochs=1,callbacks=[model_checkpoint_callback])
model.load_weights(checkpoint_filepath)
print(f"Test acc: {model.evaluate(binary_1gram_test_ds)[1]:.3f}")

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 20000)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 16)                  │         320,016 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 16)                  │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │              17 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 320,033 (1.22 MB)

 Trainable params: 320,033 (1.22 MB)

 Non-trainable params: 0 (0.00 B)

625/625 ━━━━━━━━━━━━━━━━━━━━ 502s 798ms/step - accuracy: 0.7817 - loss: 0.4779 - val_accuracy: 0.8870 - val_loss: 0.2856


ValueError: File format not supported: filepath=/tmp/checkpoint. Keras 3 only supports V3 `.keras` and `.weights.h5` files, or legacy V1/V2 `.h5` files.

"********************************************************"-
****O código a Seguir refere-se a "*TextVectorization_bag_bigram"****

---

TheTextVectorizationlayer can be configured to return arbitrary N-grams: bigrams,trigrams, etc. Just pass anngrams=Nargument

In [ ]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode="multi_hot",
)

Training and testing the binary bigram model

In [ ]:
import tensorflow as tf
text_only_train_ds = train_ds.map(lambda x, y: x)  #Prepare a dataset that only yields raw text inputs (no labels).
text_vectorization.adapt(text_only_train_ds)
binary_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorization(x), y),num_parallel_calls=4)
binary_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorization(x), y),num_parallel_calls=4)
binary_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorization(x), y),num_parallel_calls=4)
model = get_model()
model.summary()

checkpoint_filepath = '/tmp/checkpoint'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='/tmp/checkpoint.weights.h5',
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

#callbacks = [
#    keras.callbacks.ModelCheckpoint("binary_2gram.keras",save_best_only=True)
#]

model.fit(binary_2gram_train_ds.cache(),
validation_data=binary_2gram_val_ds.cache(),
epochs=1,callbacks=[model_checkpoint_callback]
)
#model = keras.models.load_model("binary_2gram.keras")
model.load_weights(checkpoint_filepath)
print(f"Test acc: {model.evaluate(binary_2gram_test_ds)[1]:.3f}")


Model: "model_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 20000)]           0         
                                                                 
 dense_18 (Dense)            (None, 16)                320016    
                                                                 
 dropout_9 (Dropout)         (None, 16)                0         
                                                                 
 dense_19 (Dense)            (None, 1)                 17        
                                                                 
Total params: 320033 (1.22 MB)
Trainable params: 320033 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/10
625/625 [==============================] - 4s 5ms/step - loss: 0.4156 - accuracy: 0.8248 - val_loss: 0.2975 - val_accuracy: 0.8830
Epoch 2/10
62

In [ ]:
print(f"Test acc: {model.evaluate(binary_2gram_val_ds)[1]:.3f}")


157/157 [==============================] - 1s 3ms/step - loss: 0.2925 - accuracy: 0.8902
Test acc: 0.890


***Tarefa para os alunos: implementar com tf_idf**